# Obtención de datos mediante Web Scrapping
(web Ayuntamiento de Madrid)

[Enlace portal web Ayuntamiento de Madrid Actividades Infantiles](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

Alternativa próximas actividades:
[Enlace portal web Ayuntamiento PRÓXIMAS actividades](https://www.madrid.es/portales/munimadrid/es/AyuntamientodeMadrid/Proximas-actividades-infantiles/?vgnextoid=cff47c52ba910710VgnVCM2000001f4a900aRCRD&vgnextchannel=2af809f68134b010VgnVCM1000000b205a0aRCRD&page=0=)
(en este aparecen muy pocas)

voy a realizar Webscrapping con Beatutiful Soup.
preguntando a las iAs:

ChatGPT: Sí, es razonable, y puedes hacerlo con BeautifulSoup + requests, si el contenido está en el HTML inicial (sin JS dinámico). Pero si cargan los planes con JavaScript, quizá necesites Selenium o Playwright.

Copilot: Factibilidad técnica
El scraping dependerá de la estructura del HTML del sitio. Por ejemplo:
Identifica las etiquetas HTML en las que se encuentran los datos (puedes inspeccionar el sitio web con herramientas como DevTools de tu navegador).
Utiliza Beautiful Soup para parsear el contenido y extraer los elementos que necesitas.

## Probando webscrapping todas las páginas
Ahora voy a adaptar el notebook anterior para todas las páginas que aparezcan en la web de cabecera

Adaptando el código para tomar todos los datos de todas las paginas que existan con actividades. Este número variará según la fecha de la consulta.

In [ ]:
# código mejorado obtener datos de pag1 en sucesivos print
import requests
from bs4 import BeautifulSoup

# URL de la página 1
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# URL web próximas actividades, prueba para ver si funciona igual
#url = "https://www.madrid.es/portales/munimadrid/es/AyuntamientodeMadrid/Proximas-actividades-infantiles/?vgnextoid=cff47c52ba910710VgnVCM2000001f4a900aRCRD&vgnextchannel=2af809f68134b010VgnVCM1000000b205a0aRCRD&page=0"

response = requests.get(url)

if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    # Extraer el número total de actividades
    total_results_container = soup.find('ul', id='totalResultsUL')
    total_actividades = 0

    if total_results_container:
        total_elementos = total_results_container.find('strong')
        if total_elementos:
            total_actividades = int(total_elementos.text)
            print(f"Número total de actividades: {total_actividades}")
        else:
            print("No se encontró el elemento que contiene el número total")
    else:
        print("No se encontró el contenedor de resultados totales")
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Se encontraron {len(actividades)} actividades:")
        print("*" * 20)  # Separador
        
        for actividad in actividades:
            # Extraer el título
            elemento = actividad.find('a', class_='event-link')
            titulo = elemento.text.strip() if elemento else "Sin título"
            # Extraer la url de la actividad
            url = elemento['href'] if elemento else "Sin enlace url"
            
            # Extraer el tipo de actividad (si existe)
            # <p class="event-type">
            elemento_tipo = actividad.find('p', class_='event-type')
            tipo = elemento_tipo.text.strip() if elemento_tipo else "Sin tipo"
            
            # Extraer las fechas
            # <p class="event-date">
            elemento_fecha = actividad.find('p', class_='event-date')
            fechas = elemento_fecha.text.strip() if elemento_fecha else "Sin fechas"
            
            # Extraer el lugar
            # <a href="******* class=event-location ">
            elemento_lugar = actividad.find('a', class_='event-location')
            lugar = elemento_lugar.text.strip() if elemento_lugar else "Sin lugar"
        
            # Imprimir los resultados en columnas
            print(f"Título: {titulo}")
            print(f"Enlace: {url}")
            print(f"Tipo: {tipo}")
            print(f"Fechas: {fechas}")
            print(f"Lugar: {lugar}")
            print("-" * 50)  # Separador entre actividades
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

In [ ]:
# en el avión, código ejemplo iterar paginas. 
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD"

# Calcular el número total de páginas (25 actividades por página)
n = total_activities // 25
total_pages = (total_activities // 25)+1
print(f"Número total de páginas a recorrer: {total_pages}")

# Recorrer todas las páginas
for i in range(0,total_pages):
    # Pagina 0, principal
    if i == 0:
        # no tiene paginación
        page_url = url
        print(f"\nAccediendo a la página cabecera: {page_url}")
    else:
        # Añadir pagina al final de las URL principal
        page_url = f'{url}&page={i}'
        print(f"\nAccediendo a la página {i}/{n}: {page_url}")
    



In [ ]:
# OPCIÓN 1: almacenar una lista completa actividades sin mostrarlas. MUY
import requests
from bs4 import BeautifulSoup
from datetime import date

# Página general
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Primera solicitud para obtener el total
respuesta = requests.get(url)
print(f"Código de respuesta: {respuesta.status_code}")

if respuesta.status_code == 200:
    sopa = BeautifulSoup(respuesta.content, 'html.parser')
    
    # Extraer el número total de actividades
    try:
        total_actividades = int(sopa.select_one('#totalResultsUL strong').text)
        print(f"Consulta fecha {fecha_hoy}. Número total de actividades: {total_actividades}")
    except (AttributeError, ValueError):
        total_actividades = 0
        print(f"Consulta fecha {fecha_hoy}. No se pudo obtener el número total de actividades")
    
    # Calcular el número total de páginas (25 actividades por página)
    if total_actividades % 25 == 0:
        total_paginas = total_actividades // 25
    else:
        total_paginas = (total_actividades // 25) + 1
    print(f"Número total de páginas a recorrer: {total_paginas}")
    
    # Lista para almacenar todas las actividades
    todas_actividades = []
    
    # Procesar la primera página (ya tenemos la respuesta)
    contenedor_principal = sopa.find('ul', class_='events-results docs')
    if contenedor_principal:
        elementos_actividad = contenedor_principal.find_all('div', class_='event-info')
        todas_actividades.extend(elementos_actividad)
        print(f"Se encontraron {len(elementos_actividad)} actividades en la página 0")
    
    # Recorrer el resto de páginas (desde 1 hasta total_paginas-1)
    for i in range(1, total_paginas):
        # Añadir página al final de la URL principal
        url_pagina = f'{url}&page={i}'
        print(f"\nAccediendo a la página {i}/{total_paginas-1}: {url_pagina}")
        
        respuesta_pagina = requests.get(url_pagina)
        
        if respuesta_pagina.status_code == 200:
            sopa_pagina = BeautifulSoup(respuesta_pagina.content, 'html.parser')
            
            contenedor_pagina = sopa_pagina.find('ul', class_='events-results docs')
            if contenedor_pagina:
                actividades_pagina = contenedor_pagina.find_all('div', class_='event-info')
                todas_actividades.extend(actividades_pagina)
                print(f"Se encontraron {len(actividades_pagina)} actividades en la página {i}")
            else:
                print(f"No se encontró el contenedor principal en la página {i}")
        else:
            print(f"Error: el servidor respondió con código {respuesta_pagina.status_code} para la página {i}.")
    
    print(f"\nTotal de actividades recopiladas: {len(todas_actividades)}")
    
    # Mostrar el primer elemento como ejemplo
    if todas_actividades:
        print("\nPrimer elemento (como muestra):")
        print(todas_actividades[0].prettify())
else:
    print(f"Error: el servidor respondió con código {respuesta.status_code}.")

In [24]:
# OPCIÓN 2: guarda la lista y muestra resultados ejemplo
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import os

# URL base
url_base = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD"

# Obtener fecha actual
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Primera solicitud para obtener el total
response = requests.get(url_base)

if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extraer el número total de actividades
    contenedor_principal = soup.find('ul', id='totalResultsUL')
    total_actividades = 0
    if contenedor_principal:
        total_elementos = contenedor_principal.find('strong')
        if total_elementos:
            total_actividades = int(total_elementos.text)
            print(f"Consulta fecha {fecha_hoy}. Número total de actividades: {total_actividades}")
        else:
            print("No se encontró el elemento que contiene el número total")
    else:
        print("No se encontró el contenedor de resultados totales")
    
    # Calcular número total de páginas
    if total_actividades % 25 == 0:
        total_paginas = total_actividades // 25
    else:
        total_paginas = (total_actividades // 25) + 1
    print(f"Número total de páginas a recorrer: {total_paginas}")
    
    # Lista para almacenar todas las actividades
    todas_actividades = []
    
    # Recorrer todas las páginas
    for pagina in range(0, total_paginas):
        url_pagina = f"{url_base}&page={pagina}"
        print(f"\nAccediendo a la página {pagina}/{total_paginas}: {url_pagina}")
        
        respuesta_pagina = requests.get(url_pagina)
        
        if respuesta_pagina.status_code == 200:
            sopa_pagina = BeautifulSoup(respuesta_pagina.text, 'html.parser')
            contenedor_principal = sopa_pagina.find('ul', class_='events-results docs')
            
            if contenedor_principal:
                # Buscar todas las actividades dentro del contenedor
                actividades_pagina = contenedor_principal.find_all('div', class_='event-info')
                print(f"Se encontraron {len(actividades_pagina)} actividades en la página {pagina}")
                
                # Extraer detalles de cada actividad
                for actividad in actividades_pagina:
                    info_actividad = {}
                    
                    # Extraer el título y URL
                    elemento = actividad.find('a', class_='event-link')
                    info_actividad['titulo'] = elemento.text.strip() if elemento else "Sin título"
                    info_actividad['url'] = elemento['href'] if elemento else "Sin enlace url"
                    
                    # Extraer el tipo de actividad
                    elemento_tipo = actividad.find('p', class_='event-type')
                    info_actividad['tipo'] = elemento_tipo.text.strip() if elemento_tipo else "Sin tipo"
                    
                    # Extraer las fechas
                    elemento_fecha = actividad.find('p', class_='event-date')
                    info_actividad['fechas'] = elemento_fecha.text.strip() if elemento_fecha else "Sin fechas"
                    
                    # Extraer el lugar
                    elemento_lugar = actividad.find('a', class_='event-location')
                    info_actividad['lugar'] = elemento_lugar.text.strip() if elemento_lugar else "Sin lugar"
                    
                    # Añadir a la lista de todas las actividades
                    todas_actividades.append(info_actividad)
            else:
                print("No se encontró el contenedor principal en esta página")
        else:
            print(f"Error: el servidor respondió con código {respuesta_pagina.status_code} para la página {pagina}.")
    
    # Mostrar resumen de datos recolectados
    print("\n" + "="*60)
    print(f"RESUMEN: Se han recolectado {len(todas_actividades)} actividades de un total de {total_actividades}")
    print("="*60)
    
    # Opcionalmente, mostrar algunos ejemplos
    print("\nPrimeras 3 actividades como ejemplo:")
    for i, act in enumerate(todas_actividades[:3], 1):
        print(f"\nActividad {i}:")
        print(f"Título: {act['titulo']}")
        print(f"Enlace: {act['url']}")
        print(f"Tipo: {act['tipo']}")
        print(f"Fechas: {act['fechas']}")
        print(f"Lugar: {act['lugar']}")
        print("-" * 50)
    
    # Crear DataFrame con las actividades
    df = pd.DataFrame(todas_actividades)
    
    # Definir ruta del archivo
    directorio = '../data/raw/'
    nombre_archivo = f'actividades_infantiles_madrid_{fecha_hoy}.csv'
    ruta_archivo = os.path.join(directorio, nombre_archivo)
    
    # Comprobar si el archivo ya existe
    if os.path.exists(ruta_archivo):
        print(f"¡ATENCIÓN! El archivo '{nombre_archivo}' ya existe.")
        respuesta = input("¿Desea sobrescribirlo? (s/n): ").lower()
        if respuesta == 's':
            df.to_csv(ruta_archivo, index=False, encoding='utf-8-sig')
            print(f"Datos guardados correctamente en '{nombre_archivo}'.")
        else:
            nuevo_nombre = f'actividades_infantiles_madrid_{fecha_hoy}_nuevo.csv'
            nueva_ruta = os.path.join(directorio, nuevo_nombre)
            df.to_csv(nueva_ruta, index=False, encoding='utf-8-sig')
            print(f"Datos guardados con nuevo nombre: '{nuevo_nombre}'.")
    else:
        df.to_csv(ruta_archivo, index=False, encoding='utf-8-sig')
        print(f"Datos guardados correctamente en '{nombre_archivo}'.")
     
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Consulta fecha 19-04-2025. Número total de actividades: 228
Número total de páginas a recorrer: 10

Accediendo a la página 0/10: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=0
Se encontraron 25 actividades en la página 0

Accediendo a la página 1/10: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1
Se encontraron 25 actividades en la página 1

Accediendo a la página 2/10: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c2

He comprobado que la web de [Enlace portal web Ayuntamiento PRÓXIMAS Actividades infantiles](https://www.madrid.es/portales/munimadrid/es/AyuntamientodeMadrid/Proximas-actividades-infantiles/?vgnextoid=cff47c52ba910710VgnVCM2000001f4a900aRCRD&vgnextchannel=2af809f68134b010VgnVCM1000000b205a0aRCRD&page=0=) contiene actividades ya incluidas en la anterior.

## Trabajando en el csv obtenido

Ahora voy a desglosar algunos datos y organizarlos hasta lograr un Dataframe más manejable y fácil de visualizar. Para ello voy a incorporar guardar el archivo obtenido en un csv.

In [25]:
# seleccionar la fecha del csv
import pandas as pd
# Definir ruta del archivo
directorio = '../data/raw/'
nombre_archivo_trabajo = 'actividades_infantiles_madrid_19-04-2025.csv'
ruta_archivo_trabajo = os.path.join(directorio, nombre_archivo_trabajo)

df = pd.read_csv(ruta_archivo_trabajo)

In [26]:
df.sample(2)

,titulo,url,tipo,fechas,lugar
29,LUDOTECAS,/portales/munimadrid/es/Inicio/Cultura-ocio-y-...,CURSOS Y TALLERES,Del sábado 5 de abril de 2025\nal sábado 26 de...,Centro Sociocultural Valverde (Fuencarral - El...
182,Asalto a la lectura. Animales grandes,/portales/munimadrid/es/Inicio/Cultura-ocio-y-...,"RECITALES, PRESENTACIONES Y ACTOS LITERARIOS, ...",Sábado 10 de mayo de 2025,Centro Cultural Clara del Rey - Museo ABC (Cen...


In [27]:
df.shape

(228, 5)

In [28]:
#  procesado de columnas. Feature Engineer

# 1. Separar el distrito del lugar y modificar lugar
def procesar_lugar(lugar):
    if isinstance(lugar, str) and '.' in lugar:
        partes = lugar.split('.')
        # Devuelve una tupla (lugar_modificado, distrito)
        return partes[0].strip(), partes[-1].strip()
    return lugar, "Sin distrito"

# Aplicar la función y desempaquetar los resultados
df['lugar_2'], df['distrito'] = zip(*df['lugar'].apply(procesar_lugar))
# Reemplazar la columna original con el nuevo valor
df['lugar'] = df['lugar_2']
# Eliminar la columna temporal
df = df.drop('lugar_2', axis=1)

# 2. Separar la columna fecha de forma simplificada
def separar_fechas(fecha):
    if not isinstance(fecha, str):
        return "Sin fecha", "Sin fecha"
    
    # Pasar a minúsculas y eliminar saltos de línea
    fecha = fecha.lower().replace('\n', ' ')
    
    # Eliminar "del " al inicio si existe
    if fecha.startswith("del "):
        fecha = fecha[4:]
    
    # Separar por "al" si existe
    if " al " in fecha:
        partes = fecha.split(" al ")
        return partes[0].strip(), partes[1].strip()
    
    # Si no hay "al", usar la misma fecha para inicio y fin
    return fecha.strip(), fecha.strip()

# Crear nuevas columnas para fecha inicio y fecha fin
#df['fecha_inicio'], df['fecha_fin'] = zip(*df['fechas'].apply(separar_fechas))

# Verificar si la columna 'fechas' existe
if 'fechas' in df.columns:
    # Crear nuevas columnas para fecha_inicio y fecha_fin
    df['fecha_inicio'], df['fecha_fin'] = zip(*df['fechas'].apply(separar_fechas))
else:
    print("La columna 'fechas' ya se elimino en el dataframe")

# 3. Añadir "www.madrid.es" al principio de las URLs
df['url'] = 'https://www.madrid.es' + df['url']
# Mover la columna 'url' al final
df['url'] = df.pop('url')



In [29]:
df.head(3)

,titulo,tipo,fechas,lugar,distrito,fecha_inicio,fecha_fin,url
0,Talleres creativos en idioma serbio,CURSOS Y TALLERES,Del domingo 22 de septiembre de 2024\nal domin...,Biblioteca Pública Municipal Iván de Vargas (C...,CENTRO,domingo 22 de septiembre de 2024,domingo 22 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
1,Tertulias en Inglés (infantil) Biblioteca Vall...,CURSOS Y TALLERES,Del miércoles 25 de septiembre de 2024\nal mié...,Biblioteca Pública Municipal Vallecas (Puente ...,PUENTE DE VALLECAS,miércoles 25 de septiembre de 2024,miércoles 11 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
2,Club de lectura (infantil) Biblioteca David Gi...,"CLUBES DE LECTURA, \r\n \r\n ...",Del lunes 30 de septiembre de 2024\nal lunes 9...,Biblioteca Pública Municipal David Gistau (Sal...,SALAMANCA,lunes 30 de septiembre de 2024,lunes 9 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...


In [38]:
# Guardar el DataFrame modificado (sin columna fechas)
if 'fechas' in df.columns:
    df.drop('fechas', axis=1, inplace=True)
    print("Columna 'fechas' eliminada del DataFrame.")
else:
    print("La columna 'fechas' no existe en el DataFrame.")

# Definir ruta del archivo a guardar:
directorio = '../data/processed/'
ruta_archivo_trabajo = os.path.join(directorio, nombre_archivo_trabajo)

df.to_csv(ruta_archivo_trabajo, index=False, encoding='utf-8-sig')
print(f"Datos procesados guardados en carpeta '{ruta_archivo_trabajo}'")

La columna 'fechas' no existe en el DataFrame.
Datos procesados guardados en carpeta '../data/processed/actividades_infantiles_madrid_19-04-2025.csv'


In [39]:
df

,titulo,tipo,lugar,distrito,fecha_inicio,fecha_fin,url
0,Talleres creativos en idioma serbio,CURSOS Y TALLERES,Biblioteca Pública Municipal Iván de Vargas (C...,CENTRO,domingo 22 de septiembre de 2024,domingo 22 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
1,Tertulias en Inglés (infantil) Biblioteca Vall...,CURSOS Y TALLERES,Biblioteca Pública Municipal Vallecas (Puente ...,PUENTE DE VALLECAS,miércoles 25 de septiembre de 2024,miércoles 11 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
2,Club de lectura (infantil) Biblioteca David Gi...,"CLUBES DE LECTURA, \r\n \r\n ...",Biblioteca Pública Municipal David Gistau (Sal...,SALAMANCA,lunes 30 de septiembre de 2024,lunes 9 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
3,Taller de cómic (Infantil) Biblioteca Pozo del...,CURSOS Y TALLERES,Biblioteca Pública Municipal Pozo del Tío Raim...,PUENTE DE VALLECAS,lunes 30 de septiembre de 2024,lunes 2 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
4,Club de lectura (infantil) Biblioteca San Fermín,"CLUBES DE LECTURA, \r\n \r\n ...",Biblioteca Pública Municipal San Fermín (Usera),USERA,miércoles 2 de octubre de 2024,miércoles 18 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
...,...,...,...,...,...,...,...
223,Campamento de verano 2025 en Puente de Vallecas,"ACTIVIDADES CALLE, ARTE URBANO, \r\n ...",Sin lugar,Sin distrito,lunes 23 de junio de 2025,viernes 5 de septiembre de 2025,https://www.madrid.es/portales/munimadrid/es/I...
224,Campamento de Verano 2025 - Distrito de Villa...,CAMPAMENTOS,Sin lugar,Sin distrito,lunes 23 de junio de 2025,jueves 31 de julio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
225,Verde que te quiero verde - Zaguán Teatro (Cas...,"CUENTACUENTOS, TÍTERES Y MARIONETAS, \r\n ...",Teatro Municipal de Títeres,RETIRO,sábado 28 de junio de 2025,domingo 29 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
226,Los más pequeños también plantamos,Sin tipo,Centro de Información y Educación Ambiental de...,RETIRO,domingo 29 de junio de 2025,domingo 29 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...


In [34]:
df.sample(10)

,titulo,tipo,lugar,distrito,fecha_inicio,fecha_fin,url
219,Planeta Tierra - Títeres Sol y Tierra (Madrid),"CUENTACUENTOS, TÍTERES Y MARIONETAS",Teatro Municipal de Títeres,RETIRO,sábado 21 de junio de 2025,domingo 22 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
108,Ronda de libros,Sin tipo,Matadero Madrid,ARGANZUELA,sábado 26 de abril de 2025,sábado 7 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
57,Entrega de premios,CONCURSOS Y CERTÁMENES,Biblioteca Pública Municipal San Blas (San Bla...,SAN BLAS-CANILLEJAS,martes 22 de abril de 2025,martes 22 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
74,'El espíritu Aloha' de Beatriz M. Cassy,"CUENTACUENTOS, TÍTERES Y MARIONETAS",Biblioteca Pública Municipal Francisco Ibáñez ...,CHAMARTIN,miércoles 23 de abril de 2025,miércoles 23 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
126,"Cuentacuentos con pictogramas: ""El enano salta...","CUENTACUENTOS, TÍTERES Y MARIONETAS, \r\n ...",Centro Cultural La Vaguada (Fuencarral - El Pa...,FUENCARRAL-EL PARDO,sábado 26 de abril de 2025,sábado 26 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
45,Taller 'Todos los animales están conectados e ...,"ITINERARIOS Y OTRAS ACTIVIDADES AMBIENTALES, \...",Aula ambiental La Cabaña del Retiro,RETIRO,sábado 19 de abril de 2025,sábado 19 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
188,Cine infantil en la Biblioteca María Lejárraga...,CINE ACTIVIDADES AUDIOVISUALES,Biblioteca Pública Municipal María Lejárraga (...,HORTALEZA,martes 13 de mayo de 2025,martes 13 de mayo de 2025,https://www.madrid.es/portales/munimadrid/es/I...
34,TALLERES FAMILIARES,"ITINERARIOS Y OTRAS ACTIVIDADES AMBIENTALES, \...",Centro de Interpretación de la Naturaleza Mont...,FUENCARRAL-EL PARDO,domingo 6 de abril de 2025,sábado 26 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
104,"CINE: ""Klaus""",Sin tipo,Centro Cultural el Torito (Moratalaz),MORATALAZ,sábado 26 de abril de 2025,sábado 26 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
44,"PAYASAS: ""Risa y carcajada: un musical de aleg...",Sin tipo,Centro Cultural el Torito (Moratalaz),MORATALAZ,sábado 19 de abril de 2025,sábado 19 de abril de 2025,https://www.madrid.es/portales/munimadrid/es/I...
